In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
import sys
import MeCab
import numpy as np
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import warnings
%matplotlib inline
warnings.simplefilter('ignore')

# スクレイピング

In [12]:
def scraping_web_page(url):
    sleep(0.5)
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

In [13]:
def scraping_lyric(urls):
    artist_dfs = []
    for url in urls:
        #曲一覧ページをスクレイピングする
        soup = scraping_web_page(url)
        #htmlをパースして曲名、各曲URL、アーティスト名、作詞、作曲者名を取得する
        contents = []
        contents.append(soup.find_all(href=re.compile('/song/\d+/$')))
        contents.append(soup.find_all(href=re.compile('/song/\d+/$')))
        contents.append(soup.find_all(class_=re.compile('td2')))
        contents.append(soup.find_all(class_=re.compile('td3')))
        contents.append(soup.find_all(class_=re.compile('td4')))
        infomations = []
        for i, content in enumerate(contents):
            tmp_list = []
            for element in content:
                if i == 0:
                    tmp_list.append(element.get('href'))
                else:
                    tmp_list.append(element.string)
            infomations.append(tmp_list)
        #DataFrameにする
        artist_df = pd.DataFrame({
            'URL': infomations[0],
            'SongName': infomations[1],
            'Artist': infomations[2],
            'Lyricist': infomations[3],
            'Composer': infomations[4]
        })
        #URLにホストネームを付加
        artist_df['URL'] = artist_df['URL'].apply(lambda x: 'https://www.uta-net.com' + x)
        artist_dfs.append(artist_df)
    df_all = pd.concat(artist_dfs)
    #各曲のページをスクレイピングする
    contents_list = []
    for i, url in tqdm(df_all['URL'].iteritems(), total=len(df_all)):
        contents_list.append(scraping_web_page(url))
    #歌詞、発売日、商品番号をdataframeに格納する
    lyrics = []
    sales_dates = []
    cd_nums = []
    for contents in contents_list:
        lyrics.append(contents.find(id='kashi_area').text)
        sales_dates.append(contents.find(id='view_amazon').text[4:14])
        cd_nums.append(contents.find(id='view_amazon').text[19:28])
    df_all['Lyric'] = lyrics
    df_all['Sales_Date'] = sales_dates
    df_all['CD_Number'] = cd_nums
    
    return df_all

In [14]:
bz_urls = [ 'https://www.uta-net.com/artist/134/', 'https://www.uta-net.com/artist/134/0/2/']
df_bz = scraping_lyric(bz_urls)
df_bz.head()

,Artist,Composer,Lyricist,SongName,URL,Lyric,Sales_Date,CD_Number
0,B'z,Takahiro Matsumoto,Kohshi Inaba,I WANNA DANCE Wicked Beat Style,https://www.uta-net.com/song/54892/,Crush! Crush! Taxi on a busy neon streetGive m...,：1990-06-2,：BMCR-900
1,B'z,松本孝弘,稲葉浩志,あいかわらずなボクら,https://www.uta-net.com/song/7938/,今まで好きなこともしたし たまに我慢もしてきた「あいつはダメだ」 なんてキミ勝手に決めないで...,：1991-11-2,：BMCR-64B
2,B'z,TAK MATSUMOTO,KOSHI INABA,愛と憎しみのハジマリ,https://www.uta-net.com/song/17979/,誰もが愛する者 守ろうとしてる頃 憎しみがスキをうかがう大破綻の雰囲気が世界中を取り巻いて朝...,：2003-09-1,：BMCV-800
3,B'z,松本孝弘,稲葉浩志,愛のバクダン,https://www.uta-net.com/song/25415/,今日も一日 不完全な世界きみが泣かなくても 誰か泣く人がこしらえた 悲しみのからくりなんとか...,：2005-03-0,：BMCV-400
4,B'z,TAK MATSUMOTO,KOSHI INABA,愛のprisoner,https://www.uta-net.com/song/26049/,"I hate you, hate you こらえきれずに夜の闇に吠える 涙流さないあなたのど...",：2000-12-0,：BMCR-704


In [15]:
glay_urls = [ 'https://www.uta-net.com/artist/356/', 'https://www.uta-net.com/artist/356/0/2/']
df_glay = scraping_lyric(glay_urls)
df_glay.head()

,Artist,Composer,Lyricist,SongName,URL,Lyric,Sales_Date,CD_Number
0,GLAY,TAKURO,TAKURO,Into the Wild,https://www.uta-net.com/song/282725/,Hey Now Hey Now知りすぎたSilly GirlHey Now Hey Now言...,：2020-03-1,：PCCN-42ポ
1,GLAY,TAKURO,TAKURO,あゝ、無常,https://www.uta-net.com/song/275020/,鏡を見つめりゃ疲れきった男が笑うかつての眼光消え失せた痩せこけたほおでシケモク並べて占えば絶...,：2019-10-0,：PCCN-38ポ
2,GLAY,HISASHI,HISASHI,アイ,https://www.uta-net.com/song/16998/,夢と希望に踊る胸が刻む鼓動をすり抜けて裏通りが誘う猥雑に心奪われた暗闇の奏でるリアルとモラル...,：1998-04-2,：PODH-702
3,GLAY,TAKURO,TAKURO,I am xxx,https://www.uta-net.com/song/80048/,いつか眠りについた後は違う私で目覚めて来るから違う時代に同じ場所で私をどうか愛して欲しいいつ...,：2009-05-2,：TOCT-402
4,GLAY,TERU,TERU,I will～,https://www.uta-net.com/song/51311/,I Believe…奇麗事を語る日々はもう過ぎて 見る景色が色を失ってゆく支え合い生きてきた...,：2007-01-3,：TOCT-261


In [16]:
mr_chi_urls = [ 'https://www.uta-net.com/artist/684/', 'https://www.uta-net.com/artist/684/0/2/']
df_mr_chi = scraping_lyric(mr_chi_urls)
df_mr_chi.head()

,Artist,Composer,Lyricist,SongName,URL,Lyric,Sales_Date,CD_Number
0,Mr.Children,桜井和寿,桜井和寿,Birthday,https://www.uta-net.com/song/282273/,しばらくして 気付いたんだ 本物だって熱くなって 冷やかして とっちらかってシャボン玉が食ら...,：2020-03-0,：TFCC-896
1,Mr.Children,桜井和寿,桜井和寿,君と重ねたモノローグ,https://www.uta-net.com/song/282272/,また会おう この道のどこかでありがとう この気持ち届くかな果てしなく続くこの時間の中でほんの...,：2020-03-0,：TFCC-896
2,Mr.Children,Kazutoshi Sakurai,Kazutoshi Sakurai,I,https://www.uta-net.com/song/105567/,もう いいでしょう!?これで終わりにしようねぇ どうでしょう!?君だってそう思うでしょ!?散...,：2010-12-0,：TFCC-863
3,Mr.Children,桜井和寿,桜井和寿,I Can Make It,https://www.uta-net.com/song/187713/,明け方 非現実的な夢をバスタブに浮かべてみる身体は疲れてるのに目は冴えるやるべきことは沢山「...,：2015-06-0,：TFCC-865
4,Mr.Children,Kazutoshi Sakurai,Kazutoshi Sakurai,I'm sorry,https://www.uta-net.com/song/24490/,あれは血迷ってた僕の思い過ごしでした八割位はあなたの言った通りでしたいましがた 人づてで 御...,：2002-07-1,：TFCC-890


歌詞部分とArtistだけ抽出

In [23]:
df_bz_extract = df_bz[['Artist', 'SongName', 'Lyric']]
print(df_bz_extract.shape)
df_bz_extract = df_bz_extract[~df_bz_extract.duplicated()]
print(df_bz_extract.shape)

(347, 3)
(346, 3)


In [24]:
df_glay_extract = df_glay[['Artist', 'SongName', 'Lyric']]
print(df_glay_extract.shape)
df_glay_extract = df_glay_extract[~df_glay_extract.duplicated()]
print(df_glay_extract.shape)

(296, 3)
(296, 3)


In [25]:
df_mr_chi_extract = df_mr_chi[['Artist', 'SongName', 'Lyric']]
print(df_mr_chi_extract.shape)
df_mr_chi_extract = df_mr_chi_extract[~df_mr_chi_extract.duplicated()]
print(df_mr_chi_extract.shape)

(235, 3)
(235, 3)


In [81]:
# 連結
df_bz_extract['target'] = 0
df_glay_extract['target'] = 1
df_mr_chi_extract['target'] = 2

df_all = pd.concat([df_bz_extract, df_glay_extract, df_mr_chi_extract])
df_all.reset_index(inplace=True)
df_all.head(), df_all.tail()

(   index Artist                         SongName  \
 0      0    B'z  I WANNA DANCE Wicked Beat Style   
 1      1    B'z                       あいかわらずなボクら   
 2      2    B'z                       愛と憎しみのハジマリ   
 3      3    B'z                           愛のバクダン   
 4      4    B'z                       愛のprisoner   
 
                                                Lyric  target  
 0  Crush! Crush! Taxi on a busy neon streetGive m...       0  
 1  今まで好きなこともしたし たまに我慢もしてきた「あいつはダメだ」 なんてキミ勝手に決めないで...       0  
 2  誰もが愛する者　守ろうとしてる頃　憎しみがスキをうかがう大破綻の雰囲気が世界中を取り巻いて朝...       0  
 3  今日も一日　不完全な世界きみが泣かなくても　誰か泣く人がこしらえた　悲しみのからくりなんとか...       0  
 4  I hate you, hate you　こらえきれずに夜の闇に吠える　涙流さないあなたのど...       0  ,
      index       Artist                    SongName  \
 872     30  Mr.Children  ROLLIN' ROLLING ～一見は百聞に如かず   
 873     31  Mr.Children                       忘れ得ぬ人   
 874     32  Mr.Children                       WALTZ   
 875     33  Mr.Children               one two three   
 876     34  Mr

In [82]:
df_all.shape

(877, 5)

In [83]:
df_all[['Lyric', 'target']].to_csv('data_all.csv', index=False)

In [84]:
N = len(df_all)
df_test = df_all.sample(int(N*0.2))
df_train = df_all.drop(df_test.index)
df_test.shape, df_train.shape

((175, 5), (702, 5))

In [85]:
df_test[['Lyric', 'target']].to_csv('test.csv', index=False, header=False,)
df_train[['Lyric', 'target']].to_csv('train.csv', index=False, header=False,)